# ニューラルネットワーク（ANN）

### ライブラリのインポート

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.9.2'

## Part 1 - データの前処理

### データセットのインポート

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [4]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [5]:
print(y)

[1 0 1 ... 1 1 0]


### カテゴリ変数のエンコーディング

 "Gender"列のエンコーディング

In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [7]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


 "Geography" 列のエンコーディング

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [9]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### フィーチャースケーリング

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [11]:
print(X)

[[ 0.99720391 -0.57873591 -0.57380915 ...  0.64609167  0.97024255
   0.02188649]
 [-1.00280393 -0.57873591  1.74273971 ... -1.54776799  0.97024255
   0.21653375]
 [ 0.99720391 -0.57873591 -0.57380915 ...  0.64609167 -1.03067011
   0.2406869 ]
 ...
 [ 0.99720391 -0.57873591 -0.57380915 ... -1.54776799  0.97024255
  -1.00864308]
 [-1.00280393  1.72790383 -0.57380915 ...  0.64609167 -1.03067011
  -0.12523071]
 [ 0.99720391 -0.57873591 -0.57380915 ...  0.64609167 -1.03067011
  -1.07636976]]


### 訓練用とテスト用へのデータセットの分割

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

## Part 2 - ANNの構築

### ANNのイニシャライズ

In [13]:
ann = tf.keras.models.Sequential()

### 入力層と、一つ目の隠れ層の追加

In [14]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### 二つ目の隠れ層の追加

In [15]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### 出力層の追加

In [16]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## ANNの訓練（学習）

### ANNのコンパイル

In [17]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### 訓練用データセットを使ったANNの訓練

In [18]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 2s 4ms/step - loss: 0.5380 - accuracy: 0.7754
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4635 - accuracy: 0.7966
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4437 - accuracy: 0.7997
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4350 - accuracy: 0.8081
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4282 - accuracy: 0.8151
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4208 - accuracy: 0.8194
Epoch 7/100
250/250 [==============================] - 1s 5ms/step - loss: 0.4140 - accuracy: 0.8235
Epoch 8/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4070 - accuracy: 0.8269
Epoch 9/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3991 - accuracy: 0.8319
Epoch 10/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3914 - accura

## Part 4 - 結果の予測とモデルの評価

### テストデータによる結果の予測

In [19]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 0s 1ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### 混同行列の作成

In [20]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1518   77]
 [ 198  207]]
